In [1]:
import pandas as pd
import numpy as np

In [4]:
property_data = pd.read_csv('datasets/property_data_set.csv')

In [3]:
print("First 10 rows of the dataset:")
property_data.head(10)

First 10 rows of the dataset:


,property_id,type,activation_date,bathroom,floor,total_floor,furnishing,gym,latitude,longitude,...,lift,locality,parking,property_age,property_size,swimming_pool,pin_code,rent,deposit,building_type
0,ff808081469fd6e20146a5af948000ea,BHK2,09-03-2017 14:36,1,3,4.0,SEMI_FURNISHED,1,12.876174,77.596571,...,1,Hulimavu,BOTH,2,850,1,560076.0,12000,120000,AP
1,ff8080814702d3d10147068359d200cd,BHK2,07-03-2017 12:02,2,4,11.0,SEMI_FURNISHED,1,13.018444,77.678122,...,1,Ramamurthy Nagar,BOTH,1,1233,1,560016.0,20000,150000,AP
2,ff808081470c645401470fb03f5800a6,BHK2,10-03-2017 13:43,2,0,4.0,NOT_FURNISHED,1,12.975072,77.665865,...,1,GM Palya,FOUR_WHEELER,0,1200,0,560075.0,15000,75000,AP
3,ff808081470c6454014715eaa5960281,BHK3,09-03-2017 22:16,2,3,4.0,SEMI_FURNISHED,0,12.888169,77.591282,...,0,Arakere,BOTH,1,1300,0,560076.0,17000,150000,AP
4,ff808081474aa867014771a0298f0aa6,BHK1,15-03-2017 18:29,1,1,2.0,SEMI_FURNISHED,0,12.990243,77.712962,...,0,Hoodi,BOTH,4,450,0,560048.0,6500,40000,IF
5,ff8080814780f30801478d14132f03f4,BHK3,23-03-2017 16:13,2,2,2.0,SEMI_FURNISHED,1,12.936557,77.558291,...,0,Banashankari,TWO_WHEELER,8,1200,1,560050.0,17500,200000,IF
6,ff8080814780f308014797c79000082e,BHK2,06-03-2017 21:30,1,1,3.0,SEMI_FURNISHED,1,12.930229,77.633311,...,1,Koramangala,TWO_WHEELER,5,600,1,560034.0,20000,100000,IF
7,ff80808147a95f0a0147ab0d88fa0080,BHK2,05-03-2017 18:12,2,0,5.0,SEMI_FURNISHED,1,12.992509,77.658957,...,1,Bennigana Halli,BOTH,18,800,1,560016.0,18000,120000,AP
8,ff80808147ac6af10147afc279480099,BHK3,07-02-2017 14:05,3,0,2.0,SEMI_FURNISHED,1,12.979944,77.538875,...,0,Vijayanagar,BOTH,15,2000,1,560040.0,25000,250000,IF
9,ff80808147cbdc2a0147cf8577880153,BHK2,12-03-2017 11:21,1,1,2.0,NOT_FURNISHED,1,12.958654,77.717208,...,0,Kundalahalli,TWO_WHEELER,8,600,0,560037.0,12000,40000,AP


In [5]:
print("Summary Stats:")
property_data.describe()

Summary Stats:


,bathroom,floor,total_floor,gym,latitude,longitude,lift,property_age,property_size,swimming_pool,pin_code,rent,deposit
count,28888.000000,28888.000000,28845.000000,28888.000000,28888.000000,28888.000000,28888.000000,28888.000000,28888.000000,28888.000000,2.856300e+04,28888.000000,2.888800e+04
mean,1.795867,1.786659,3.518807,0.165951,12.957859,77.605186,0.279009,4.380539,966.338999,0.142585,5.615600e+05,14430.768797,1.233257e+05
std,0.778676,2.361768,3.362536,0.372043,0.219500,0.921003,0.448520,7.458914,619.390016,0.349655,8.979452e+04,9055.133718,1.255028e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1000.000000,0.000000e+00
25%,1.000000,0.000000,2.000000,0.000000,12.906324,77.563295,0.000000,0.000000,600.000000,0.000000,5.600400e+05,8000.000000,5.000000e+04
50%,2.000000,1.000000,3.000000,0.000000,12.953939,77.614568,0.000000,3.000000,900.000000,0.000000,5.600670e+05,12500.000000,1.000000e+05
75%,2.000000,2.000000,4.000000,0.000000,13.009369,77.669235,1.000000,6.000000,1200.000000,0.000000,5.600840e+05,18000.000000,1.500000e+05
max,22.000000,123.000000,38.000000,1.000000,21.176277,80.256293,1.000000,800.000000,40000.000000,1.000000,5.601000e+06,75000.000000,4.200000e+06


In [7]:
print("Information of the dataset:")
property_data.info()

Information of the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28888 entries, 0 to 28887
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_id      28888 non-null  object 
 1   type             28888 non-null  object 
 2   activation_date  28888 non-null  object 
 3   bathroom         28888 non-null  int64  
 4   floor            28888 non-null  int64  
 5   total_floor      28845 non-null  float64
 6   furnishing       28888 non-null  object 
 7   gym              28888 non-null  int64  
 8   latitude         28888 non-null  float64
 9   longitude        28888 non-null  float64
 10  lease_type       28888 non-null  object 
 11  lift             28888 non-null  int64  
 12  locality         28888 non-null  object 
 13  parking          28888 non-null  object 
 14  property_age     28888 non-null  int64  
 15  property_size    28888 non-null  int64  
 16  swimming_pool    28888 non-nul

In [10]:
print("Missing values in dataset by columns:")
property_data.isnull().sum()

Missing values in dataset by columns:


property_id          0
type                 0
activation_date      0
bathroom             0
floor                0
total_floor         43
furnishing           0
gym                  0
latitude             0
longitude            0
lease_type           0
lift                 0
locality             0
parking              0
property_age         0
property_size        0
swimming_pool        0
pin_code           325
rent                 0
deposit              0
building_type       79
dtype: int64